In [7]:
import os
os.chdir("../")
%pwd

'd:\\projects\\MLOps\\Sign-Language-Recognition'

In [8]:
from dataclasses import dataclass
from pathlib import Path

In [10]:
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    raw_data_path: Path

In [11]:
from Sign_Language.constants import *
from Sign_Language.utils.common import create_directories, read_yaml

In [12]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            train_data_path=config.train_data_path,
            test_data_path=config.test_data_path,
            raw_data_path=config.raw_data_path 
        )

        return data_ingestion_config

In [17]:
import os
import pandas as pd
from Sign_Language import logger
from Sign_Language.utils.common import get_size
from sklearn.model_selection import train_test_split

In [19]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def load_file(self):
        if not os.path.exists(self.config.raw_data_path):
            df = pd.read_csv("research/data/Data.csv")
            logger.info(f"Read the Raw dataset as DataFrame: {self.config.raw_data_path}")

            df.to_csv(self.config.raw_data_path, index=True, header=True)

            # traindf, testdf = train_test_split(df, test_size=0.2, random_state=12)

            # traindf.to_csv(self.config.train_data_path, index=True, header=True)
            # testdf.to_csv(self.config.test_data_path, index=True, header=True)

            logger.info(f"loaded the raw dataset: {self.config.raw_data_path}")

        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.raw_data_path))}")

    def create_train_test_data(self):
        if not os.path.exists(self.config.train_data_path):
            df = pd.read_csv(self.config.raw_data_path)
            traindf, testdf = train_test_split(df, test_size=0.2, random_state=12)

            traindf.to_csv(self.config.train_data_path, index=True, header=True)
            testdf.to_csv(self.config.test_data_path, index=True, header=True)

            logger.info("Created the training and test data")
        else:
            logger.info(f"File already exists of size: \n \
                        {get_size(Path(self.config.train_data_path))} \n\
                        {get_size(Path(self.config.train_data_path))}")

In [21]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.load_file()
    data_ingestion.create_train_test_data()
except Exception as e:
    raise e

[2024-01-23 19:27:15,391: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-01-23 19:27:15,397: INFO: common: yaml file: params.yaml loaded successfully]
[2024-01-23 19:27:15,440: INFO: common: created directory at: artifacts]
[2024-01-23 19:27:15,449: INFO: common: created directory at: artifacts/data-ingestion]
[2024-01-23 19:27:17,111: INFO: 2911108747: Read the Raw dataset as DataFrame: artifacts/data-ingestion/data.csv]
[2024-01-23 19:27:23,407: INFO: 2911108747: loaded the raw dataset: artifacts/data-ingestion/data.csv]
[2024-01-23 19:27:31,020: INFO: 2911108747: Created the training and test data]
